# Exporting&nbsp;RasterFrames

In [1]:
import astraea.spark.rasterframes._
import geotrellis.spark._
import geotrellis.raster._
import geotrellis.raster.render._
import geotrellis.raster.io.geotiff.SinglebandGeoTiff
import org.apache.spark.sql._
import org.apache.spark.sql.functions._

implicit val spark = SparkSession.builder().config("spark.ui.enabled", "false").
  master("local[*]").appName("RasterFrames").getOrCreate().withRasterFrames
spark.sparkContext.setLogLevel("ERROR")
import spark.implicits._
val scene = SinglebandGeoTiff("../samples/L8-B8-Robinson-IL.tiff")
val rf = scene.projectedRaster.toRF(128, 128).cache()

Intitializing Scala interpreter ...

Spark Web UI available at http://172.18.0.2:4041
SparkContext available as 'sc' (version = 2.2.0, master = local[*], app id = local-1531158623967)
SparkSession available as 'spark'


import astraea.spark.rasterframes._
import geotrellis.spark._
import geotrellis.raster._
import geotrellis.raster.render._
import geotrellis.raster.io.geotiff.SinglebandGeoTiff
import org.apache.spark.sql._
import org.apache.spark.sql.functions._
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@4c8fae5b
import spark.implicits._
scene: geotrellis.raster.io.geotiff.SinglebandGeoTiff = SinglebandGeoTiff(geotrellis.raster.UShortConstantNoDataArrayTile@2c1b12ca,Extent(431902.5, 4313647.5, 443512.5, 4321147.5),EPSG:32616,Tags(Map(AREA_OR_POINT -> POINT),List(Map())),GeoTiffOptions(geotrellis.raster.io.geotiff.Striped@46a546c2,geotrellis.raster.io.geotiff.compression.DeflateCompression$@32dc819e,1,None))
rf: org.apache.spark.sql.DataFrame with shapeless.tag.Tagged[a...

While the goal of RasterFrames is to make it as easy as possible to do your geospatial analysis with a single 
construct, it is helpful to be able to transform it into other representations for various use cases.

## Converting to Array

The cell values within a `Tile` are encoded internally as an array. There may be use cases 
where the additional context provided by the `Tile` construct is no longer needed and one would
prefer to work with the underlying array data.

The @scaladoc[`tileToArray`][tileToArray] column function requires a type parameter to indicate the array element
type you would like used. The following types may be used: `Int`, `Double`, `Byte`, `Short`, `Float`

In [11]:
val withArrays = rf.withColumn("tileData", tileToArray[Short]($"tile")).drop("tile")
withArrays.show(5, 40)

+-----------+----------------------------------------+
|spatial_key|                                tileData|
+-----------+----------------------------------------+
|      [6,3]|[11576, 12488, 9070, 9614, 10561, 112...|
|      [4,0]|[10155, 9839, 9296, 9495, 9921, 9962,...|
|      [2,1]|[14375, 13830, 12012, 11623, 11248, 1...|
|      [0,3]|[10825, 10328, 10289, 10467, 10495, 1...|
|      [0,0]|[14294, 14277, 13939, 13604, 14182, 1...|
+-----------+----------------------------------------+
only showing top 5 rows



withArrays: org.apache.spark.sql.DataFrame = [spatial_key: struct<col: int, row: int>, tileData: array<smallint>]


You can convert the data back to an array, but you have to specify the target tile dimensions. 

In [3]:
val tileBack = withArrays.withColumn("tileAgain", arrayToTile($"tileData", 128, 128))
tileBack.drop("tileData").show(5, 40)

+-----------+--------------------------------------+
|spatial_key|                             tileAgain|
+-----------+--------------------------------------+
|      [6,3]|ShortRawArrayTile([S@30ea44fc,128,128)|
|      [4,0]|ShortRawArrayTile([S@7f7bd257,128,128)|
|      [2,1]| ShortRawArrayTile([S@6a5c727,128,128)|
|      [0,3]|ShortRawArrayTile([S@429d1b3c,128,128)|
|      [0,0]|ShortRawArrayTile([S@20b26a37,128,128)|
+-----------+--------------------------------------+
only showing top 5 rows



tileBack: org.apache.spark.sql.DataFrame = [spatial_key: struct<col: int, row: int>, tileData: array<smallint> ... 1 more field]


Note that the created tile will not have a `NoData` value associated with it. Here's how you can do that:

In [4]:
val tileBackAgain = withArrays.withColumn("tileAgain", withNoData(arrayToTile($"tileData", 128, 128), 3))
tileBackAgain.drop("tileData").show(5, 50)

+-----------+--------------------------------------------------+
|spatial_key|                                         tileAgain|
+-----------+--------------------------------------------------+
|      [6,3]|ShortUserDefinedNoDataArrayTile([S@433a4961,128...|
|      [4,0]|ShortUserDefinedNoDataArrayTile([S@2b926fa3,128...|
|      [2,1]|ShortUserDefinedNoDataArrayTile([S@7b5e92f8,128...|
|      [0,3]|ShortUserDefinedNoDataArrayTile([S@2df6100d,128...|
|      [0,0]|ShortUserDefinedNoDataArrayTile([S@3c162593,128...|
+-----------+--------------------------------------------------+
only showing top 5 rows



tileBackAgain: org.apache.spark.sql.DataFrame = [spatial_key: struct<col: int, row: int>, tileData: array<smallint> ... 1 more field]


## Writing to Parquet

It is often useful to write Spark results in a form that is easily reloaded for subsequent analysis. 
The [Parquet](https://parquet.apache.org/)columnar storage format, native to Spark, is ideal for this. RasterFrames
work just like any other DataFrame in this scenario as long as `rfInit` is called to register
the imagery types.


Let's assume we have a RasterFrame we've done some fancy processing on: 

In [5]:
import geotrellis.raster.equalization._
val equalizer = udf((t: Tile) => t.equalize())
val equalized = rf.withColumn("equalized", equalizer($"tile")).asRF

equalized.printSchema
equalized.select(aggStats($"tile")).show(false)
equalized.select(aggStats($"equalized")).show(false)

root
 |-- spatial_key: struct (nullable = true)
 |    |-- col: integer (nullable = false)
 |    |-- row: integer (nullable = false)
 |-- tile: rf_tile (nullable = true)
 |-- equalized: rf_tile (nullable = true)

+---------+------+-------+-----------------+------------------+
|dataCells|min   |max    |mean             |variance          |
+---------+------+-------+-----------------+------------------+
|387000   |7209.0|39217.0|10160.48549870801|3315238.5311127007|
+---------+------+-------+-----------------+------------------+

+---------+---+-------+------------------+-------------------+
|dataCells|min|max    |mean              |variance           |
+---------+---+-------+------------------+-------------------+
|458724   |4.0|65535.0|32763.474431248418|3.025128587936561E8|
+---------+---+-------+------------------+-------------------+



import geotrellis.raster.equalization._
equalizer: org.apache.spark.sql.expressions.UserDefinedFunction = UserDefinedFunction(<function1>,org.apache.spark.sql.gt.types.TileUDT@4efee117,Some(List(org.apache.spark.sql.gt.types.TileUDT@4efee117)))
equalized: astraea.spark.rasterframes.RasterFrame = [spatial_key: struct<col: int, row: int>, tile: rf_tile ... 1 more field]


We write it out just like any other DataFrame, including the ability to specify partitioning:

In [6]:
val filePath = "/tmp/equalized.parquet"
equalized.select("*", "spatial_key.*").write.partitionBy("col", "row").mode(SaveMode.Overwrite).parquet(filePath)

filePath: String = /tmp/equalized.parquet


Let's confirm partitioning happened as expected:

In [7]:
import java.io.File
new File(filePath).list.filter(f => !f.contains("_"))

import java.io.File
res5: Array[String] = Array(col=5, col=1, col=6, col=3, col=4, col=2, col=0)


Now we can load the data back in and check it out:

In [8]:
val rf2 = spark.read.parquet(filePath)

rf2.printSchema
equalized.select(aggStats($"tile")).show(false)
equalized.select(aggStats($"equalized")).show(false)

root
 |-- spatial_key: struct (nullable = true)
 |    |-- col: integer (nullable = true)
 |    |-- row: integer (nullable = true)
 |-- tile: rf_tile (nullable = true)
 |-- equalized: rf_tile (nullable = true)
 |-- col: integer (nullable = true)
 |-- row: integer (nullable = true)

+---------+------+-------+-----------------+------------------+
|dataCells|min   |max    |mean             |variance          |
+---------+------+-------+-----------------+------------------+
|387000   |7209.0|39217.0|10160.48549870801|3315238.5311127007|
+---------+------+-------+-----------------+------------------+

+---------+---+-------+------------------+-------------------+
|dataCells|min|max    |mean              |variance           |
+---------+---+-------+------------------+-------------------+
|458724   |4.0|65535.0|32763.474431248418|3.025128587936561E8|
+---------+---+-------+------------------+-------------------+



rf2: org.apache.spark.sql.DataFrame = [spatial_key: struct<col: int, row: int>, tile: rf_tile ... 3 more fields]


## Exporting a Raster

For the purposes of debugging, the RasterFrame tiles can be reassembled back into a raster for viewing. However, 
keep in mind that this will download all the data to the driver, and reassemble it in-memory. So it's not appropriate 
for very large coverages.

Here's how one might render the image to a georeferenced GeoTIFF file: 

In [9]:
import geotrellis.raster.io.geotiff.GeoTiff
val image = equalized.toRaster($"equalized", 774, 500)

import geotrellis.raster.io.geotiff.GeoTiff
image: geotrellis.raster.ProjectedRaster[geotrellis.raster.Tile] = ProjectedRaster(Raster(CroppedTile(geotrellis.raster.UShortConstantNoDataArrayTile@243db6ea,GridBounds(0,0,773,499)),Extent(431902.5, 4313647.5, 443512.5, 4321147.5)),utm-CS)


In [10]:
GeoTiff(image).write("/tmp/rf-raster.tiff")

Here's how one might render a raster frame to a false color PNG file:

In [12]:
val colors = ColorMap.fromQuantileBreaks(image.tile.histogram, ColorRamps.BlueToOrange)
image.tile.color(colors).renderPng().write("/tmp/rf-raster.png")

colors: geotrellis.raster.render.ColorMap = geotrellis.raster.render.IntColorMap@15c2cef1


![-](outputs/rf-raster.png)

## Exporting to a GeoTrellis Layer

For future analysis it is helpful to persist a RasterFrame as a [GeoTrellis layer](http://geotrellis.readthedocs.io/en/latest/guide/tile-backends.html).

First, convert the RasterFrame into a TileLayerRDD. The return type is an Either;
the `left` side is for spatial-only keyed data

In [13]:
val tlRDD = equalized.toTileLayerRDD($"equalized").left.get

tlRDD: geotrellis.spark.TileLayerRDD[geotrellis.spark.SpatialKey] = ContextRDD[73] at RDD at ContextRDD.scala:35


Then create a GeoTrellis layer writer:

In [14]:
import java.nio.file.Files
import spray.json._
import DefaultJsonProtocol._
import geotrellis.spark.io._
val p = Files.createTempDirectory("gt-store")
val writer: LayerWriter[LayerId] = LayerWriter(p.toUri)

val layerId = LayerId("equalized", 0)
writer.write(layerId, tlRDD, index.ZCurveKeyIndexMethod)

import java.nio.file.Files
import spray.json._
import DefaultJsonProtocol._
import geotrellis.spark.io._
p: java.nio.file.Path = /tmp/gt-store7655057995775895561
writer: geotrellis.spark.io.LayerWriter[geotrellis.spark.LayerId] = geotrellis.spark.io.file.FileLayerWriter@430733c5
layerId: geotrellis.spark.LayerId = Layer(name = "equalized", zoom = 0)


Take a look at the metadata in JSON format:

In [15]:
AttributeStore(p.toUri).readMetadata[JsValue](layerId).prettyPrint

res11: String =
{
  "extent": {
    "xmin": 431902.5,
    "ymin": 4313647.5,
    "xmax": 443512.5,
    "ymax": 4321147.5
  },
  "layoutDefinition": {
    "extent": {
      "xmin": 431902.5,
      "ymin": 4313467.5,
      "xmax": 445342.5,
      "ymax": 4321147.5
    },
    "tileLayout": {
      "layoutCols": 7,
      "layoutRows": 4,
      "tileCols": 128,
      "tileRows": 128
    }
  },
  "bounds": {
    "minKey": {
      "col": 0,
      "row": 0
    },
    "maxKey": {
      "col": 6,
      "row": 3
    }
  },
  "cellType": "uint16",
  "crs": "+proj=utm +zone=16 +datum=WGS84 +units=m +no_defs "
}


## Converting to `RDD` and `TileLayerRDD`

Since a `RasterFrame` is just a `DataFrame` with extra metadata, the method 
`DataFrame.rdd` is available for simple conversion back to `RDD` space. The type returned 
by `.rdd` is dependent upon how you select it.

In [16]:
import scala.reflect.runtime.universe._
def showType[T: TypeTag](t: T) = println(implicitly[TypeTag[T]].tpe.toString)

showType(rf.rdd)

showType(rf.select(rf.spatialKeyColumn, $"tile".as[Tile]).rdd) 

showType(rf.select(rf.spatialKeyColumn, $"tile").as[(SpatialKey, Tile)].rdd) 

org.apache.spark.rdd.RDD[org.apache.spark.sql.Row]
org.apache.spark.rdd.RDD[(geotrellis.spark.SpatialKey, geotrellis.raster.Tile)]
org.apache.spark.rdd.RDD[(geotrellis.spark.SpatialKey, geotrellis.raster.Tile)]


import scala.reflect.runtime.universe._
showType: [T](t: T)(implicit evidence$1: reflect.runtime.universe.TypeTag[T])Unit


If your goal convert a single tile column with its spatial key back to a `TileLayerRDD[K]`, then there's an additional
extension method on `RasterFrame` called `toTileLayerRDD`, which preserves the tile layer metadata,
enhancing interoperation with GeoTrellis RDD extension methods.

In [17]:
showType(rf.toTileLayerRDD($"tile".as[Tile]))

scala.Either[geotrellis.spark.TileLayerRDD[geotrellis.spark.SpatialKey],geotrellis.spark.TileLayerRDD[geotrellis.spark.SpaceTimeKey]]


[rfInit]: astraea.spark.rasterframes.package#rfInit%28SQLContext%29:Unit
[rdd]: org.apache.spark.sql.Dataset#frdd:org.apache.spark.rdd.RDD[T]
[toTileLayerRDD]: astraea.spark.rasterframes.RasterFrameMethods#toTileLayerRDD%28tileCol:RasterFrameMethods.this.TileColumn%29:Either[geotrellis.spark.TileLayerRDD[geotrellis.spark.SpatialKey],geotrellis.spark.TileLayerRDD[geotrellis.spark.SpaceTimeKey]]
[tileToArray]: astraea.spark.rasterframes.ColumnFunctions#tileToArray